# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

#Local Edition

In [1]:
# No API keys required; everything runs via Ollama locally

import os
import json
import requests
import textwrap
import subprocess
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display




In [2]:
# Ollama configuration
MODEL = "llama3.2"
OLLAMA_BASE = "http://localhost:11434/api/chat"

print(f"Ollama Model: {MODEL}")
print(f"Ollama Endpoint: {OLLAMA_BASE}")

Ollama Model: llama3.2
Ollama Endpoint: http://localhost:11434/api/chat


In [3]:
# === Utility Functions: Fetch Website Links and Content ===

def fetch_website_links(url):
    """Fetch all hyperlinks from a webpage."""
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        links = []
        for tag in soup.find_all("a", href=True):
            href = tag["href"]
            full_url = urljoin(url, href)
            if full_url.startswith("http"):
                links.append(full_url)
        return list(set(links))  # remove duplicates
    except Exception as e:
        print(f"Error fetching links from {url}: {e}")
        return []

def fetch_website_contents(url):
    """Fetch and clean text content from a webpage."""
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        for tag in soup(["script", "style", "noscript"]):
            tag.extract()
        text = soup.get_text(separator=" ", strip=True)
        return text[:2000]  # keep content concise
    except Exception as e:
        print(f"Error fetching contents from {url}: {e}")
        return ""


In [4]:
# === Ollama Chat Function ===

def ollama_chat(prompt, model=MODEL):
    """
    Send a prompt to Ollama and return the model's response.
    """
    try:
        response = requests.post(
            OLLAMA_BASE,
            json={
                "model": model,
                "messages": [{"role": "user", "content": prompt}],
                "stream": False,
            },
            timeout=60,
        )
        if response.status_code != 200:
            print("Ollama returned an error:", response.text)
            return ""
        data = response.json()
        return data.get("message", {}).get("content", "")
    except Exception as e:
        print("Error connecting to Ollama — ensure it's running via `ollama serve`.")
        print("Error:", e)
        return ""


In [5]:
# === Test Ollama Connection ===

test_prompt = "Say hello! This is a quick test of local Ollama."
reply = ollama_chat(test_prompt)
display(Markdown(f"**Ollama says:** {reply}"))


**Ollama says:** Hello! It's nice to meet you, even if it's just for a quick test of our chat functionality. How's your day going so far?

In [6]:
# --- Summarize a single webpage ---
def summarize_page_with_llama(url, max_chars=4000):
    """
    Fetches a webpage, extracts readable text, and summarizes it locally.
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator=" ", strip=True)
        text = " ".join(text.split())[:max_chars]

        # Ask local Llama for summarization
        prompt = f"Summarize the following webpage clearly and concisely:\n\n{text}"
        result = subprocess.run(
            ["ollama", "run", MODEL_LLAMA],
            input=prompt.encode("utf-8"),
            capture_output=True,
            text=True
        )

        summary = result.stdout.strip()
        return f"Summary of {url}:\n{summary}"

    except Exception as e:
        return f"Error summarizing {url}: {e}"


In [7]:

# --- MODEL NAME ---
MODEL_LLAMA = "llama3"  # change this to your installed Ollama model

# --- Step 1: Summarize a single webpage ---
def summarize_page_with_llama(url, max_chars=4000):
    """
    Fetches a webpage, extracts main text, and summarizes it.
    Currently uses a simple text truncation for demo purposes.
    Replace with an API call to Llama for actual summarization.
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(separator=" ", strip=True)
        text = " ".join(text.split())[:max_chars]

        # --- Optional: use local Llama summarization instead of textwrap ---
        result = subprocess.run(
            ["ollama", "run", MODEL_LLAMA],
            input=f"Summarize this webpage:\n\n{text}".encode("utf-8"),
            capture_output=True,
            text=True
        )
        summary = result.stdout.strip() if result.stdout.strip() else textwrap.shorten(text, width=400, placeholder="...")

        return f"Summary of {url}:\n{summary}"

    except Exception as e:
        return f"Error summarizing {url}: {e}"

In [8]:
# --- Step 2: Combine multiple summaries into one brochure prompt ---
def create_company_brochure(company_name, urls):
    combined_summary = ""

    for url in urls:
        section = summarize_page_with_llama(url)
        combined_summary += f"\n\n### Source: {url}\n\n{section}"

    brochure_prompt = f"""
You are a helpful assistant that writes professional company brochures.
Based on the following summaries of the company's web pages, write a cohesive
and engaging brochure about **{company_name}**.

{combined_summary}
"""
    display(Markdown("#Brochure Prompt Preview"))
    display(Markdown(brochure_prompt))
    return brochure_prompt


In [9]:
# --- Step 3: Generate brochure from the prompt ---
def generate_brochure_from_prompt(prompt):
    result = subprocess.run(
        ["ollama", "run", MODEL_LLAMA],
        input=prompt.encode("utf-8"),
        capture_output=True,
        text=True
    )

    brochure_text = result.stdout.strip()
    display(Markdown("#Company Brochure"))
    display(Markdown(brochure_text))
    return brochure_text


In [10]:
# --- Step 4: Run the flow ---
company = "Hugging Face"
urls = [
    "https://huggingface.co",
    "https://huggingface.co/about",
    "https://huggingface.co/jobs"
]

brochure_prompt = create_company_brochure(company, urls)


#Brochure Prompt Preview


You are a helpful assistant that writes professional company brochures.
Based on the following summaries of the company's web pages, write a cohesive
and engaging brochure about **Hugging Face**.



### Source: https://huggingface.co

Error summarizing https://huggingface.co: write() argument must be str, not bytes

### Source: https://huggingface.co/about

Error summarizing https://huggingface.co/about: write() argument must be str, not bytes

### Source: https://huggingface.co/jobs

Error summarizing https://huggingface.co/jobs: write() argument must be str, not bytes


In [ ]:
# --- Stream brochure generation from Llama (Ollama) with UTF-8 safety ---

def generate_brochure_from_prompt(prompt):
    """Stream brochure generation from local Llama (Ollama) so output appears live (UTF-8 safe)."""
    import subprocess
    from IPython.display import Markdown, display, update_display

    display_handle = display(Markdown("Generating brochure... please wait."), display_id=True)

    # Start Ollama process with UTF-8 encoding
    process = subprocess.Popen(
        ["ollama", "run", MODEL_LLAMA],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        encoding="utf-8",          # ensures UTF-8 decoding
        errors="replace"           # replaces invalid bytes instead of crashing
    )

    # Send the prompt to Llama
    process.stdin.write(prompt)
    process.stdin.close()

    output = ""
    for line in process.stdout:
        output += line
        update_display(Markdown(output), display_id=display_handle.display_id)

    process.wait()
    brochure_text = output.strip()

    display(Markdown("#Company Brochure"))
    display(Markdown(brochure_text))
    return brochure_text


# --- Run it on the prompt generated earlier ---
brochure_text = generate_brochure_from_prompt(brochure_prompt)


[?2026h[?25l[1Gpulling manifest ⠋ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠙ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠹ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠸ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠼ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠴ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠦ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠧ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠇ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠏ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠋ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest ⠙ [K[?25h[?2026l[?2026h[?25l[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s   20m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s   20m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s   20m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s   20m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s   20m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s   20m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s   20m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s   20m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  19m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  19m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.4 MB/s    5m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.3 MB/s   5m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  898 KB/s   7m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  786 KB/s   8m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  815 KB/s   8m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  932 KB/s   7m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  990 KB/s   6m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  1.0 MB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s  12m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  564 KB/s   12m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  582 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  524 KB/s   13m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  393 KB/s  17m20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  14m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  19m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s  14m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s  14m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s  14m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s  14m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s  14m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s  14m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s   14m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s   14m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s   14m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  473 KB/s   14m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s    9m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s    9m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s    9m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s   8m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s   8m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s   8m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s   8m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s   8m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s   8m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  736 KB/s   8m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s    8m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m15s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m15s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s    5m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   6m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  924 KB/s    7m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  691 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  458 KB/s   14m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  342 KB/s  18m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  18m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  27m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  55m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   18 MB/s     19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   16 MB/s     23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   14 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB   12 MB/s     30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  9.8 MB/s     38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.7 MB/s     48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.1 MB/s    1m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.1 MB/s   1m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  2.5 MB/s   2m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  701 KB/s   8m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  261 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  13m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s   13m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s   13m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s   13m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s   13m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s   13m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s   13m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s   13m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s   9m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  17m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  51m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  51m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  51m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  116 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  116 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  116 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  116 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  116 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  116 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  116 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB  116 KB/s  59m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  662 KB/s    9m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s  10m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   8m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   7m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  6.9 MB/s     51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.6 MB/s    1m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  4.2 MB/s   1m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.4 MB/s   4m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  385 KB/s  15m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  16m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  25m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB  116 KB/s  50m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  116 KB/s   1h16m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  91% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  92% ▕████████████████  ▏ 4.3 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   20 MB/s     16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   19 MB/s     18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   17 MB/s     20s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   15 MB/s     22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   13 MB/s     25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB   10 MB/s     32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  7.9 MB/s     42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  5.2 MB/s    1m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  3.2 MB/s   1m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.3 MB/s   4m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.2 MB/s   4m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  1.0 MB/s   5m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  349 KB/s  15m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  326 KB/s  16m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  233 KB/s  23m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  444 KB/s  12m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  699 KB/s   7m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  791 KB/s   6m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  932 KB/s   5m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  815 KB/s   6m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  604 KB/s   8m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  582 KB/s   9m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  93% ▕████████████████  ▏ 4.3 GB/4.7 GB  466 KB/s  11m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB  466 KB/s  18m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.5 MB/s    1m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  6.3 MB/s   1m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  3.7 MB/s    2m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  865 KB/s   8m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m15s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s  32m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  305 KB/s  24m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  233 KB/s   32m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  466 KB/s  15m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  349 KB/s  21m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  276 KB/s  26m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  398 KB/s  18m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  282 KB/s  26m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  90% ▕████████████████  ▏ 4.2 GB/4.7 GB  165 KB/s  44m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB  165 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.0 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB                  [K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.3 MB/s    3m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.7 MB/s   2m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  3.6 MB/s   2m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  2.8 MB/s   3m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s  13m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  628 KB/s  15m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  635 KB/s  15m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  29m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  264 KB/s   37m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  320 KB/s  30m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s   32m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s  31m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s  31m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s  31m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s  31m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s  31m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s  31m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  305 KB/s  31m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  415 KB/s  23m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  23m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  354 KB/s  27m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m50s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m49s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  25m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s   36m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  87% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  24m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  613 KB/s  15m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  620 KB/s  15m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s  11m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   13m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  618 KB/s  15m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  19m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  19m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  19m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  19m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  19m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   19m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   19m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   19m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   19m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   19m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   19m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  398 KB/s   24m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  384 KB/s   25m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  35m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s  59m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s  59m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s   59m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s   59m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s   59m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s   59m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s   59m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s   59m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s   59m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  162 KB/s   59m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  59m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h2m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  34m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  25m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  24m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s   23m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s  22m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s  22m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s  22m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s  22m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  413 KB/s  22m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m6s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  672 KB/s   14m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  966 KB/s   9m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  844 KB/s  11m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  851 KB/s   11m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  611 KB/s  15m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  584 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  332 KB/s  28m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h1m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  58m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s  56m13s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  167 KB/s   56m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  33m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m15s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  32m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  225 KB/s  41m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  334 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  349 KB/s  26m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  356 KB/s  26m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  247 KB/s  37m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  480 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  497 KB/s  18m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s   16m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s   16m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s   16m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s   16m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  575 KB/s   16m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  648 KB/s  14m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  538 KB/s  17m14s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  524 KB/s  17m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  517 KB/s  17m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m38s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  23m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m19s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  34m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m58s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  329 KB/s  27m57s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m2s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  400 KB/s   23m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  407 KB/s  22m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  283 KB/s  32m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  174 KB/s  52m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m27s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  222 KB/s  41m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  223 KB/s  41m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  57m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m37s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  171 KB/s  53m28s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m23s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  375 KB/s  24m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  23m42s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m12s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m11s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  18m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s  16m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s   16m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  564 KB/s   16m7s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  722 KB/s  12m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  728 KB/s  12m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s  12m21s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  709 KB/s  12m48s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  512 KB/s  17m44s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m5s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s   18m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m4s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s   23m3s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  267 KB/s  33m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  262 KB/s  34m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  33m35s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  59m10s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s   59m8s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  58m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  58m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  58m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  58m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  58m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  291 KB/s  30m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m33s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m30s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  276 KB/s  32m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m51s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m24s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  462 KB/s  19m22s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m18s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m17s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  626 KB/s  14m16s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  742 KB/s   12m1s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  735 KB/s   12m9s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m47s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  502 KB/s  17m46s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  88% ▕███████████████   ▏ 4.1 GB/4.7 GB  509 KB/s  17m31s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  324 KB/s  27m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m45s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m43s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m41s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m40s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m39s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m36s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m32s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m29s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  160 KB/s  55m25s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  393 KB/s  22m34s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s   23m0s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  385 KB/s  22m59s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  378 KB/s  23m26s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m56s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  269 KB/s  32m52s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m55s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  145 KB/s    1h0m[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m54s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K
pulling 6a0746a1ec1a:  89% ▕███████████████   ▏ 4.1 GB/4.7 GB  152 KB/s  57m53s[K[?25h[?2026l[?2026h[?25l[A[1Gpulling manifest [K


# With API Keys

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [ ]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [ ]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [ ]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [ ]:
select_relevant_links("https://edwarddonner.com")

AuthenticationError: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [ ]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [ ]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [ ]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [ ]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanImage-3.0
Updated
about 3 hours ago
•
825
•
714
deepseek-ai/DeepSeek-V3.2-Exp
Updated
3 days ago
•
10.7k
•
470
tencent/Hunyuan3D-Part
Updated
4 days ago
•
2.57k
•
461
zai-org/GLM-4.6
Updated
2 days ago
•
9.72k
•
298
inclusionAI/Ring-1T-preview
Updated
1 day ago
•
818
•
208
Browse 1M+ models
Spaces
Running
1.29k
1.29k
Wan2.2 Animate
👁
Wan2.2 Animate
Running
14.5k
14.5k
DeepSite v3
🐳
Generate any application with DeepSeek
Running
on
Zero
MCP
1.46k
1.46k
Wan2.2 14B Fast
🎥
generate a video from an image with a text prompt


In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


"\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanImage-3.0\nUpdated\nabout 4 hours ago\n•\n825\n•\n714\ndeepseek-ai/DeepSeek-V3.2-Exp\nUpdated\n3 days ago\n•\n10.7k\n•\n470\ntencent/Hunyuan3D-Part\nUpdated\n4 days ago\n•\n2.57k\n•\n461\nzai-org/GLM-4.6\nUpdated\n2 days ago\n•\n9.72k\n•\n298\ninclusionAI/Ring-1T-preview\nUpdated\n1 day ago\n•\n818\n•\n208\nBrowse 1M+ models\nSpaces\nRunning\n1.29k\n1.29k\nWan2.2 Animate\n👁\n

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the premier collaboration platform for the machine learning (ML) community, dedicated to building the future of artificial intelligence through openness and shared knowledge. It serves as a vibrant hub where ML engineers, scientists, and enthusiasts from across the globe can share, discover, and experiment with cutting-edge open-source machine learning models, datasets, and applications.

At its core, Hugging Face empowers the next generation of AI innovators to learn, collaborate, and contribute to an open and ethical AI future.

---

## What We Offer

- **Model Hub**: Access and browse over **1 million pre-trained models** covering all modalities including text, image, video, audio, and 3D.
- **Datasets**: Explore and use more than **250,000 datasets** curated for various machine learning tasks.
- **Spaces**: Deploy and share interactive ML applications and demos; over **400,000 apps** available and growing.
- **Compute Solutions**: Scalable paid compute services including optimized inference endpoints and GPU support starting at $0.60/hour.
- **Enterprise Solutions**: Customized offerings tailored to accelerate business adoption of machine learning technologies.
- **Open Source Stack**: Leverage the extensive Hugging Face open-source libraries to move faster in development.

---

## Our Community

Hugging Face is much more than a platform — it is a thriving, fast-growing community of machine learning practitioners who collectively build, maintain, and democratize AI resources. Collaborators range from individual researchers to top organizations, contributing models like:

- Tencent’s HunyuanImage series
- deepseek-ai’s DeepSeek models
- NVIDIA and Facebook’s datasets and models

Community members benefit from the ability to:

- Host and collaborate on unlimited public projects
- Share their work globally to build a professional ML portfolio
- Collaborate using state-of-the-art open-source tools  
 
---

## Company Culture

- **Collaborative:** Empowering users to co-create and improve AI together.
- **Open & Ethical:** Committed to building an open, transparent, and socially responsible AI ecosystem.
- **Innovative:** Constantly advancing the frontier in machine learning through community-driven innovation.
- **Inclusive:** Bringing together diverse contributors and users worldwide.

---

## Careers at Hugging Face

Join a team on the front lines of democratizing AI and machine learning! Hugging Face is looking for passionate engineers, researchers, data scientists, and product experts who want to make a global impact by enabling accessible, ethical, and collaborative AI development.

By working here, you'll:

- Collaborate with world-class ML experts
- Shape the future of open-source AI tools
- Impact a vibrant global community of hundreds of thousands of users  
- Thrive in an open culture that values creativity, diversity, and ethical AI  

Visit [huggingface.co/careers](https://huggingface.co/careers) to explore current job openings.

---

## Why Choose Hugging Face?

- The **largest and most diverse** collection of open-source machine learning resources.
- A **community-first platform** that fosters collaboration and rapid innovation.
- Flexible solutions for individuals, communities, startups, and enterprises.
- Trusted by industry leaders and researchers worldwide.

---

## Get Started Today

- Explore AI apps and models at [huggingface.co](https://huggingface.co)
- Join the community, build your portfolio, and accelerate your AI journey.
- Scale your projects with our pay-as-you-go compute resources.

**Hugging Face** — The AI Community Building the Future.

---

# Contact & Links

- Website: https://huggingface.co
- Community: https://huggingface.co/community
- Documentation: https://huggingface.co/docs
- Careers: https://huggingface.co/careers

---

*Hugging Face is your home for collaboration, innovation, and advancing machine learning together.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a vibrant and fast-growing AI and machine learning community dedicated to advancing the future of open and ethical AI. It serves as a collaboration platform where machine learning engineers, scientists, and enthusiasts from around the world create, discover, and share machine learning models, datasets, and applications.

The Hugging Face Hub is the central place for hosting and collaborating on over **1 million models**, **250k+ datasets**, and **400k+ applications** spanning multiple modalities including text, images, video, audio, and 3D.

---

## What Hugging Face Offers

- **Models:** Browse and explore a vast collection of AI models, updated frequently by a diverse community. Popular trending models include image generators, video editors, and language models.
  
- **Datasets:** Access a rich catalog of datasets ranging from natural language prompts to persona datasets and beyond, enabling rapid model training and experimentation.
  
- **Spaces:** Deploy, share, and run AI applications on Hugging Face's platform, including innovative tools such as video generation from images, AI toolkits, and interactive applications.

- **Compute:** Easily accelerate your machine learning workloads with optimized inference endpoints and GPU-backed applications starting as low as $0.60/hour.

- **Enterprise Solutions:** Hugging Face provides tailored enterprise offerings to help organizations harness AI technology securely and effectively at scale.

---

## Company Culture

Hugging Face fosters an **open, collaborative, and ethical AI community** — empowering developers and researchers to work together transparently and inclusively. The culture emphasizes:

- **Community-driven innovation:** Encouraging open-source contributions and sharing knowledge.
- **Ethical AI development:** Committed to building responsible AI systems for the benefit of all.
- **Diversity and inclusion:** Supporting a broad and global community in building AI tools and resources.
- **Learning and growth:** Offering a platform for users to build their machine learning portfolios and grow their expertise.

---

## Customers & Community

Hugging Face serves a global community that includes:

- Machine learning researchers and practitioners
- Developers building innovative AI applications
- Enterprises seeking scalable AI solutions
- AI enthusiasts and learners who want to experiment and grow their skills

With over a million models and hundreds of thousands of datasets and applications, the platform connects users with cutting-edge AI technologies from companies like Tencent, NVIDIA, Facebook, and OpenAI — showcasing the power of open collaboration.

---

## Careers at Hugging Face

Interested in joining Hugging Face? The company values talented individuals passionate about AI, open-source technologies, and community building. Opportunities include roles in:

- Machine learning research & engineering
- Software development
- Cloud infrastructure and compute optimization
- Community management and developer relations
- Enterprise solutions & sales

At Hugging Face, you will work at the forefront of AI innovation while contributing to a global, open, and ethical AI movement.

---

## Get Involved

- **Explore AI models and datasets:** Visit [huggingface.co](https://huggingface.co) to browse millions of resources.
- **Build and share your own models and apps:** Create your portfolio in an open, collaborative environment.
- **Accelerate your projects:** Utilize Hugging Face’s Compute and Enterprise services.
- **Join the community:** Engage with thousands of practitioners working toward the future of AI.

---

### Contact & Follow

- Website: [https://huggingface.co](https://huggingface.co)
- Join the community to learn, collaborate, and innovate in AI!

---

*Hugging Face — Empowering the next generation of machine learning for a better, open future.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>